In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.io import read_image
from torchvision.transforms import (
    Compose,
    Normalize,
    Resize,
    ToPILImage,
    ToTensor,
)
import torchvision

from tqdm import tqdm
from datetime import datetime

In [2]:
class PlankFilter(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.fc1 = nn.Linear(in_features=128 * 30 * 30, out_features=128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 128 * 30 * 30)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.sigmoid(x)


In [3]:
model = PlankFilter()
model.load_state_dict(torch.load("./plank_filter.pt"))
model.eval()

PlankFilter(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=115200, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)

## Evaluate on cameras

In [9]:
camera_path1 = [
    "./camera_01_unnested/",
    "./camera_02_unnested/",
    "./camera_03_unnested/",
    "./camera_04_unnested/",
    "./camera_05_unnested/",
    "./camera_06_unnested/",
    "./camera_07_unnested/",
]

camera_path2 = [
    "./camera_08_unnested/",
    "./camera_09_unnested/",
    "./camera_10_unnested/",
    "./camera_11_unnested/",
    "./camera_12_unnested/",
    "./camera_13_unnested/",
    "./camera_14_unnested/",
    "./camera_15_unnested/",
    "./camera_16_unnested/"
]

img_pipeline = Compose(
    [
        ToPILImage(),
        Resize((256, 256)),
        ToTensor(),
    ]
)


In [15]:
results = []

for i, cam_path in tqdm(enumerate(camera_path2,8)):
    print(f"{'':_>3} Processing camera {i} {'':_>3}")
    print(f"{datetime.now():%H:%M}")
    for j, file in enumerate(os.listdir(cam_path)):

        if j % 5000 == 0:
            print(f"Processing Image {j: >10}")
        if file.endswith(".jpg"):
            #print(cam_path + file, j)
            img = read_image(cam_path + file)
            img = img_pipeline(img)
            img = img.unsqueeze(0)
            with torch.no_grad():
                pred = model(img)
            results.append(dict(
                camera = i,
                file = file,
                plank = np.round(pred.item())
            ))


0it [00:00, ?it/s]

___ Processing camera 8 ___
13:55
Processing Image          0
Processing Image       5000
Processing Image      10000
Processing Image      15000
Processing Image      20000
Processing Image      25000
Processing Image      30000
Processing Image      35000
Processing Image      40000
Processing Image      45000
Processing Image      50000
Processing Image      55000


1it [32:27, 1947.98s/it]

___ Processing camera 9 ___
14:27
Processing Image          0
Processing Image       5000
Processing Image      10000
Processing Image      15000
Processing Image      20000
Processing Image      25000
Processing Image      30000
Processing Image      35000
Processing Image      40000
Processing Image      45000
Processing Image      50000


2it [1:01:12, 1816.79s/it]

...

___ Processing camera 11 ___
15:26
Processing Image          0
Processing Image       5000
Processing Image      10000
Processing Image      15000
Processing Image      20000
Processing Image      25000
Processing Image      30000
Processing Image      35000
Processing Image      40000
Processing Image      45000
Processing Image      50000


4it [1:59:43, 1770.21s/it]

___ Processing camera 12 ___
15:55
Processing Image          0
Processing Image       5000
Processing Image      10000
Processing Image      15000
Processing Image      20000
Processing Image      25000
Processing Image      30000


4it [2:15:25, 2031.49s/it]


RuntimeError: Image is incomplete or truncated

In [16]:
df = pd.DataFrame(results)
df

,camera,file,plank
0,8,2017-03-17T16:49:49.jpg,1.0
1,8,2017-03-11T16:54:36.jpg,1.0
2,8,2017-03-31T05:34:06.jpg,1.0
3,8,2017-04-13T14:58:13.jpg,1.0
4,8,2017-03-29T17:38:49.jpg,1.0
...,...,...,...
245442,12,2017-04-17T11:34:29.jpg,1.0
245443,12,2017-04-16T09:59:34.jpg,1.0
245444,12,2017-04-16T12:51:21.jpg,1.0
245445,12,2017-02-28T08:38:23.jpg,1.0


In [17]:
df.to_csv("plank_results8-11.csv", index=False)